In [1]:
!cp /content/drive/MyDrive/c3i/MachineLearningCSV.zip .

In [2]:
!unzip MachineLearningCSV.zip

Archive:  MachineLearningCSV.zip
   creating: MachineLearningCVE/
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  


In [19]:
import os
import numpy as np
dir = "/content/MachineLearningCVE/"
a = os.listdir(dir)
import pandas as pd
li = []

for filename in a:
    df = pd.read_csv(dir+filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)


In [4]:
frame

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,155,2,2,88,120,44,44,44.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,59317,110,1,1,0,0,0,0,0.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,53,166,2,2,88,188,44,44,44.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,54726,81,1,1,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import os
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve,auc,roc_curve
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score, matthews_corrcoef, average_precision_score, roc_auc_score, precision_recall_curve, auc, roc_curve

In [6]:
frame.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [10]:
def load_data(df, verbose=True, train_size=50000, test_size=None):
    Y = df[" Label"].map(lambda x: 1 if (x == "BENIGN") else -1)
    labels = df[" Label"]
    df.drop(columns=[" Label", " Destination Port"], inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(df, Y, train_size=train_size, test_size=test_size, shuffle=True, stratify=Y, random_state=42)
    if verbose:
        print("***** Train Data *****")
        print(labels.loc[y_train.index].value_counts())
        print("***** Test Data *****")
        print(labels.loc[y_test.index].value_counts())
    return X_train, X_test, y_train, y_test, labels.loc[y_train.index], labels.loc[y_test.index]



In [20]:
frame.replace([np.inf, -np.inf], np.NaN, inplace=True)

In [21]:
frame.replace(np.NaN, 0, inplace=True)

In [22]:
X_train, X_test, y_train, y_test, train_labels, test_labels = load_data(frame)

***** Train Data *****
BENIGN                      40150
DoS Hulk                     4120
PortScan                     2772
DDoS                         2265
DoS GoldenEye                 179
FTP-Patator                   158
SSH-Patator                   101
DoS slowloris                  96
DoS Slowhttptest               93
Web Attack � Brute Force       26
Bot                            26
Web Attack � XSS               11
Infiltration                    2
Heartbleed                      1
Name:  Label, dtype: int64
***** Test Data *****
BENIGN                        2232947
DoS Hulk                       226953
PortScan                       156158
DDoS                           125762
DoS GoldenEye                   10114
FTP-Patator                      7780
SSH-Patator                      5796
DoS slowloris                    5700
DoS Slowhttptest                 5406
Bot                              1940
Web Attack � Brute Force         1481
Web Attack � XSS                  

In [12]:
def anomaly_scores(original, transformed):
    sse = np.sum((original - transformed)**2, axis=1)
    return sse

def evaluate_results(y_true, score):
    precision, recall, threshold = precision_recall_curve(y_true, score, pos_label=-1)
    au_precision_recall = auc(recall, precision)
    results = pd.DataFrame({'precision': precision, 'recall': recall})
    results["f1"] = 2*precision*recall/(precision+recall)
    max_index = results["f1"].idxmax()
    best = results.loc[results["f1"].idxmax()]
    best["threshold"] = threshold[max_index]
    best["au_precision_recall"] = au_precision_recall
    fpr, tpr, thresholds = roc_curve(y_true, score, pos_label=-1)
    best["auroc"] = auc(fpr, tpr)
    return best

def evaluate_predictions(y_true, y_pred):
    results = {}
    results['recall'] = recall_score(y_true, y_pred, pos_label=-1, zero_division=0)
    results['precision'] = precision_score(y_true, y_pred, pos_label=-1, zero_division=0)
    results['f1'] = f1_score(y_true, y_pred, pos_label=-1, zero_division=0)
    return results

def evaluate_test_data(y_true, score, threshold):
    y_pred = np.array([1 if score < threshold else -1 for score in score])
    results = evaluate_predictions(y_true, y_pred)
    precision, recall, threshold = precision_recall_curve(y_true, score, pos_label=-1)
    results['au_precision_recall'] = auc(recall, precision)
    fpr, tpr, thresholds = roc_curve(y_true, score, pos_label=-1)
    results["auroc"] = auc(fpr, tpr)
    return results

In [25]:
scaler = QuantileTransformer(output_distribution='normal')
X_train_s = scaler.fit_transform(X_train)
X_val, X_t, y_val, y_t, label_val, label_t = train_test_split(X_test, y_test, test_labels, train_size=0.20, random_state=42, stratify=test_labels, shuffle=True)
X_val_s = scaler.transform(X_val)

In [26]:
print("***** Train Data *****")
print(train_labels.value_counts())
print("***** Validation Data *****")
print(label_val.value_counts())
print("***** Test Data *****")
print(label_t.value_counts())

***** Train Data *****
BENIGN                      40150
DoS Hulk                     4120
PortScan                     2772
DDoS                         2265
DoS GoldenEye                 179
FTP-Patator                   158
SSH-Patator                   101
DoS slowloris                  96
DoS Slowhttptest               93
Web Attack � Brute Force       26
Bot                            26
Web Attack � XSS               11
Infiltration                    2
Heartbleed                      1
Name:  Label, dtype: int64
***** Validation Data *****
BENIGN                        446589
DoS Hulk                       45391
PortScan                       31232
DDoS                           25152
DoS GoldenEye                   2023
FTP-Patator                     1556
SSH-Patator                     1159
DoS slowloris                   1140
DoS Slowhttptest                1081
Bot                              388
Web Attack � Brute Force         296
Web Attack � XSS                 128
In

In [28]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')


{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [29]:
pca = PCA(n_components=32, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)
pca.fit(X_train_s)
x_train_feature = pca.transform(X_train_s)
x_val_feature = pca.transform(X_val_s)
#X_val_pca = pca.transform(X_val_s)
X_val_pca_inv = pca.inverse_transform(x_val_feature)
val_score_pca = anomaly_scores(X_val_s, X_val_pca_inv)
val_metrics_pca = evaluate_results(y_val, val_score_pca)


In [30]:
val_metrics_pca

precision              0.255608
recall                 0.824524
f1                     0.390239
threshold              0.205580
au_precision_recall    0.211959
auroc                  0.569458
Name: 168853, dtype: float64

In [31]:
# Isolation Forest        
forest = IsolationForest(bootstrap=False, contamination=4.161677249308696e-05, max_features=0.6674219692639616, max_samples=0.8646939376341813, random_state=42, verbose=0, warm_start=False)
        
forest.fit(x_train_feature)
y_val_pred = forest.predict(x_val_feature)
val_metrics_if = evaluate_predictions(y_val, y_val_pred)
y_val_score = forest.decision_function(x_val_feature)
val_metrics_if = evaluate_results(y_val, -y_val_score)

In [35]:
np.unique(y_val_pred,return_counts=True)

(array([-1,  1]), array([    40, 556108]))

In [32]:
val_metrics_if

precision              0.272511
recall                 0.999991
f1                     0.428303
threshold             -0.291247
au_precision_recall    0.207728
auroc                  0.572958
Name: 3543, dtype: float64

In [36]:
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM

pca = PCA(n_components=32, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)

pca = PCA(n_components=32, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)
pca.fit(X_train_s)
x_train_feature = pca.transform(X_train_s)
x_val_feature = pca.transform(X_val_s)

svm = OneClassSVM(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, nu=0.5, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
svm.fit(x_train_feature)
y_val_pred = svm.predict(x_val_feature)
val_metrics = evaluate_predictions(y_val, y_val_pred)



In [37]:
val_metrics

{'f1': 0.38086987553626983,
 'precision': 0.2657176878112146,
 'recall': 0.6721583804160316}